   # <center> Notebook for Sampling 

###  Simple model (ST- : single temperature)

### For more complex model, see: 
https://xpsi-group.github.io/xpsi

## <center> Importing modules

In [1]:
%matplotlib inline

import os
import numpy as np
import math


from matplotlib import pyplot as plt
from matplotlib import rcParams
from matplotlib.offsetbox import AnchoredText
from matplotlib.ticker import MultipleLocator, AutoLocator, AutoMinorLocator
from matplotlib import gridspec
from matplotlib import cm
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches


import xpsi
from xpsi import Parameter
from scipy.interpolate import Akima1DInterpolator
from xpsi.global_imports import _c, _G, _dpr, gravradius, _csq, _km, _2pi

/=============================================\
| X-PSI: X-ray Pulse Simulation and Inference |
|---------------------------------------------|
|                Version: 2.0.0               |
|---------------------------------------------|
|      https://xpsi-group.github.io/xpsi      |
\=============================================/

Imported GetDist version: 1.4
Imported nestcheck version: 0.2.1


# Loading data

In [2]:

data = xpsi.Data(np.loadtxt("./Data/xpsi_good_realisation.dat", dtype=np.double),
                     channels=np.arange(10,301),
                     phases=np.linspace(0.0, 1.0, 33),
                     first=0,
                     last=290,
                     exposure_time=1000.0)


Setting channels for event data...
Channels set.


# Instrument settings

In [3]:
# Let use the same fake telescope used to geenrate the synthetic data




channel_number=np.arange(0,1501)    # The channel nnumber
energy_low=np.arange(0,15.01, 0.01) # Lower bounds of each channel
energy_high=energy_low+0.01         # Upper bounds of each channel
channel_edges=np.array([list(channel_number),list(energy_low),list(energy_high)]).T

# ARF
arf_energy_low=[0.1]
arf_energy_high=[0.105]
arf_val=[1800]

counter=1
while arf_energy_low[-1]<=14.995:
    arf_energy_low.append(arf_energy_low[-1]+0.005)
    arf_energy_high.append(arf_energy_high[-1]+0.005)
    arf_val.append(1800)
    counter +=1


ARF=np.array([list(arf_energy_low),
              list(arf_energy_high),
              list(arf_val)]).T

# RMF
RMF=np.diag(np.full(counter,1))        

In [4]:
# Instrument Class

class CustomInstrument(xpsi.Instrument):
    
    """ Fake telescope response. """
    
    

    def __call__(self, signal, *args):
        """ Overwrite base just to show it is possible.

        We loaded only a submatrix of the total instrument response
        matrix into memory, so here we can simplify the method in the
        base class.

        """
        matrix = self.construct_matrix()

        self._folded_signal = np.dot(matrix, signal)

        return self._folded_signal

    @classmethod
    def from_response_files(cls, ARF, RMF, max_input, min_input=0,channel=[1,1500],
                            channel_edges=None):
        """ Constructor which converts response files into :class:`numpy.ndarray`s.
        :param str ARF: Path to ARF which is compatible with
                                :func:`numpy.loadtxt`.
        :param str RMF: Path to RMF which is compatible with
                                :func:`numpy.loadtxt`.
        :param str channel_edges: Optional path to edges which is compatible with
                                  :func:`numpy.loadtxt`.
        """

        if min_input != 0:
            min_input = int(min_input)

        max_input = int(max_input)

        matrix = np.ascontiguousarray(RMF[min_input:max_input,channel[0]:channel[1]].T, dtype=np.double)

        edges = np.zeros(ARF[min_input:max_input,2].shape[0]+1, dtype=np.double)
        
        

        edges[0] = ARF[min_input,0]; edges[1:] = ARF[min_input:max_input,1]

        for i in range(matrix.shape[0]):
            matrix[i,:] *= ARF[min_input:max_input,2]
    

        channels = np.arange(channel[0],channel[1])
    

        return cls(matrix, edges, channels, channel_edges[channel[0]:channel[1]+1,1])


In [5]:
Instrument = CustomInstrument.from_response_files(ARF =ARF,
                                             RMF = RMF,
                                             max_input = 301,
                                             min_input = 10,
                                             channel=[10,301],
                                             channel_edges =channel_edges)

Setting channels for loaded instrument response (sub)matrix...
Channels set.
No parameters supplied... empty subspace created.


# Signal

In [6]:
# Nothing very special here

import six as _six

from xpsi.likelihoods.default_background_marginalisation import eval_marginal_likelihood
from xpsi.likelihoods.default_background_marginalisation import precomputation

class CustomSignal(xpsi.Signal):
    """ A custom calculation of the logarithm of the NICER likelihood.

    We extend the :class:`xpsi.Signal.Signal` class to make it callable.

    We overwrite the body of the __call__ method. The docstring for the
    abstract method is copied.

    """

    def __init__(self, workspace_intervals = 1000, epsabs = 0, epsrel = 1.0e-8,
                 epsilon = 1.0e-3, sigmas = 10.0, support = None, *args, **kwargs):
        """ Perform precomputation. """

        super(CustomSignal, self).__init__(*args, **kwargs)

        try:
            self._precomp = precomputation(self._data.counts.astype(np.int32))
        except AttributeError:
            print('No data... can synthesise data but cannot evaluate a '
                  'likelihood function.')
        else:
            self._workspace_intervals = workspace_intervals
            self._epsabs = epsabs
            self._epsrel = epsrel
            self._epsilon = epsilon
            self._sigmas = sigmas

            if support is not None:
                self._support = support
            else:
                self._support = -1.0 * np.ones((self._data.counts.shape[0],2))
                self._support[:,0] = 0.0

    @property
    def support(self):
        return self._support

    @support.setter
    def support(self, obj):
        self._support = obj

    def __call__(self, *args, **kwargs):
        self.loglikelihood, self.expected_counts, self.background_signal,self.background_given_support = \
                eval_marginal_likelihood(self._data.exposure_time,
                                          self._data.phases,
                                          self._data.counts,
                                          self._signals,
                                          self._phases,
                                          self._shifts,
                                          self._precomp,
                                          self._support,
                                          self._workspace_intervals,
                                          self._epsabs,
                                          self._epsrel,
                                          self._epsilon,
                                          self._sigmas,
                                          kwargs.get('llzero'))


In [7]:
#from CustomSignal import CustomSignal

signal = CustomSignal(data = data,
                      instrument = Instrument,
                      interstellar = None,
                      cache = True,
                      workspace_intervals = 1000,
                      epsrel = 1.0e-8,
                      epsilon = 1.0e-3,
                      sigmas = 10.0)



Creating parameter:
    > Named "phase_shift" with fixed value 0.000e+00.
    > The phase shift for the signal, a periodic parameter [cycles].


# Space-time

### These are the parameters used to generate the synthetic data, hence for the sampling to be fast, let use some tight prior around those values

- 1.4,                         # Mass in solar Mass
- 12,                          # Equatorial radius in km
- 1.,                          # Distance in kpc
- math.cos(60*np.pi/180),      # Cosine of Earth inclination to rotation axis
- 0.0,                         # Phase shift
- 70*np.pi/180,                # Colatitude of the centre of the superseding region
- 0.75,                        # Angular radius of the (circular) superseding region
- 6.7,                         # Temperature in log 10
- -2                           # Background sprectral index : gamma (E^gamma) 


In [8]:
bounds = dict(distance = (0.5,2),
              mass = (1.0,1.6),
              radius = (10,13),
              cos_inclination = (0,1))


spacetime = xpsi.Spacetime(bounds,
                           values=dict(frequency = 314.0))


Creating parameter:
    > Named "frequency" with fixed value 3.140e+02.
    > Spin frequency [Hz].
Creating parameter:
    > Named "mass" with bounds [1.000e+00, 1.600e+00].
    > Gravitational mass [solar masses].
Creating parameter:
    > Named "radius" with bounds [1.000e+01, 1.300e+01].
    > Coordinate equatorial radius [km].
Creating parameter:
    > Named "distance" with bounds [5.000e-01, 2.000e+00].
    > Earth distance [kpc].
Creating parameter:
    > Named "cos_inclination" with bounds [0.000e+00, 1.000e+00].
    > Cosine of Earth inclination to rotation axis.


# Hot-spot

In [9]:

bounds = dict(super_colatitude = (0.001, math.pi/2 - 0.001),
              super_radius = (0.001, math.pi/2 - 0.001),
              phase_shift = (-0.25, 0.75),
              super_temperature = (6., 7.))  # Valery model limit


hot_spot = xpsi.HotRegion(bounds=bounds,
                                values={},
                                symmetry=True,
                                omit=False,
                                cede=False,
                                concentric=False,
                                sqrt_num_cells=32,
                                min_sqrt_num_cells=16,
                                max_sqrt_num_cells=64,
                                num_leaves=64,
                                num_rays=512,
                                is_antiphased=True,
                                image_order_limit=3, # up to tertiary
                                prefix='hot')


Creating parameter:
    > Named "super_colatitude" with bounds [1.000e-03, 1.570e+00].
    > The colatitude of the centre of the superseding region [radians].
Creating parameter:
    > Named "super_radius" with bounds [1.000e-03, 1.570e+00].
    > The angular radius of the (circular) superseding region [radians].
Creating parameter:
    > Named "phase_shift" with bounds [-2.500e-01, 7.500e-01].
    > The phase of the hot region, a periodic parameter [cycles].
Creating parameter:
    > Named "super_temperature" with bounds [6.000e+00, 7.000e+00].
    > log10(superseding region effective temperature [K]).


# Phostosphere

In [10]:
# Let's always use our black body model

In [11]:
class CustomPhotosphere(xpsi.Photosphere):
    """ Implement method for imaging."""

    @property
    def global_variables(self):

        return np.array([self['hot__super_colatitude'],
                          self['hot__phase_shift'] * _2pi,
                          self['hot__super_radius'],
                          self['hot__super_temperature']])



In [12]:
photosphere = CustomPhotosphere(hot = hot_spot, elsewhere = None,
                                values=dict(mode_frequency = spacetime['frequency']))



Creating parameter:
    > Named "mode_frequency" with fixed value 3.140e+02.
    > Coordinate frequency of the mode of radiative asymmetry in the
photosphere that is assumed to generate the pulsed signal [Hz].


# Star

In [13]:
star = xpsi.Star(spacetime = spacetime, photospheres = photosphere)

# Prior

In [14]:
# For fun, let put consider that we have a prior knowlege  on the distance
# Say a Gaussian centered on 1kpc with a std=0.1, troncated at +-5 sigma

# We also take define the colatitude to be uniform in cosine space


In [15]:
from scipy.interpolate import Akima1DInterpolator

from scipy.stats import truncnorm

class CustomPrior(xpsi.Prior):
    """ A custom (joint) prior distribution.

    Source: Fictitious
    Model variant: ST-

    """

    __derived_names__ = ['compactness', 'phase_separation',]

    def __init__(self):
        """ Nothing to be done.

        A direct reference to the spacetime object could be put here
        for use in __call__:

        .. code-block::

            self.spacetime = ref

        Instead we get a reference to the spacetime object through the
        a reference to a likelihood object which encapsulates a
        reference to the spacetime object.

        """
        super(CustomPrior, self).__init__() # not strictly required if no hyperparameters

    def __call__(self, p = None):
        """ Evaluate distribution at ``p``.

        :param list p: Model parameter values.

        :returns: Logarithm of the distribution evaluated at ``p``.

        """
        temp = super(CustomPrior, self).__call__(p)
        if not np.isfinite(temp):
            return temp

        # based on contemporary EOS theory
        if not self.parameters['radius'] <= 16.0:
            return -np.inf

        ref = self.parameters.star.spacetime # shortcut

        # Compactness limit
        R_p = 1.0 + ref.epsilon * (-0.788 + 1.030 * ref.zeta)
        if R_p < 1.76 / ref.R_r_s:
            return -np.inf

        mu = math.sqrt(-1.0 / (3.0 * ref.epsilon * (-0.788 + 1.030 * ref.zeta)))

        # 2-surface cross-section have a single maximum in |z|
        # i.e., an elliptical surface; minor effect on support, if any,
        # for high spin frequenies
        if mu < 1.0:
            return -np.inf

        ref = self.parameters

        return 0.0

    def inverse_sample(self, hypercube=None):
        """ Draw sample uniformly from the distribution via inverse sampling. """

        to_cache = self.parameters.vector

        if hypercube is None:
            hypercube = np.random.rand(len(self))

        # the base method is useful, so to avoid writing that code again:
        _ = super(CustomPrior, self).inverse_sample(hypercube)

        ref = self.parameters # shortcut
        
        idx = ref.index('distance')
        ref['distance'] = truncnorm.ppf(hypercube[idx], -5.0, 5.0, loc=1.0, scale=0.1)

        # flat priors in cosine of hot region centre colatitudes (isotropy)
        # support modified by no-overlap rejection condition
        idx = ref.index('hot__super_colatitude')
        a, b = ref.get_param('hot__super_colatitude').bounds
        a = math.cos(a); b = math.cos(b)
        ref['hot__super_colatitude'] = math.acos(b + (a - b) * hypercube[idx])


        # restore proper cache
        for parameter, cache in zip(ref, to_cache):
            parameter.cached = cache

        # it is important that we return the desired vector because it is
        # automatically written to disk by MultiNest and only by MultiNest
        return self.parameters.vector

    def transform(self, p, **kwargs):
        """ A transformation for post-processing. """

        p = list(p) # copy

        # used ordered names and values
        ref = dict(zip(self.parameters.names, p))

        # compactness ratio M/R_eq
        p += [gravradius(ref['mass']) / ref['radius']]

        return p


In [16]:
prior = CustomPrior()

No parameters supplied... empty subspace created.


# Likelihood

In [17]:

likelihood = xpsi.Likelihood(star = star, signals = signal,
                             num_energies = 128,
                             threads = 1,
                             externally_updated = True,
                             prior = prior)

In [18]:
likelihood

Free parameters
---------------
mass: Gravitational mass [solar masses].
radius: Coordinate equatorial radius [km].
distance: Earth distance [kpc].
cos_inclination: Cosine of Earth inclination to rotation axis.
hot__phase_shift: The phase of the hot region, a periodic parameter [cycles].
hot__super_colatitude: The colatitude of the centre of the superseding region [radians].
hot__super_radius: The angular radius of the (circular) superseding region [radians].
hot__super_temperature: log10(superseding region effective temperature [K]).

In [19]:
# Crutial step, if the likelihood check fails, then something went terrible wrong :)
p=[1.4,10,1.,math.cos(60*np.pi/180),0.0,70*np.pi/180, 0.75,6.8]

likelihood.check(None, [-47881.27817666349], 1.0e-5, physical_points=[p])

Checking likelihood and prior evaluation before commencing sampling...
Not using ``allclose`` function from NumPy.
Using fallback implementation instead.
Checking closeness of likelihood arrays:
-4.7881278177e+04 | -4.7881278177e+04 .....
Closeness evaluated.
Log-likelihood value checks passed on root process.
Checks passed.


'Log-likelihood value checks passed on root process.'

# Sampling :)

In [ ]:
# Do not run this, but instead run the main in located in modules:  mpiexec -n xxxx python main.py 

# wrapped_params = [0] * len(likelihood)
# wrapped_params[likelihood.index('hot__phase_shift')] = 1

# runtime_params = {'resume': False,
#                       'importance_nested_sampling': False,
#                       'multimodal': False,
#                       'n_clustering_params': None,
#                       'outputfiles_basename': './Outputs/ST_live_1000_eff_0.3',
#                       'n_iter_before_update': 100,
#                       'n_live_points': 1000,
#                       'sampling_efficiency': 0.3,
#                       'const_efficiency_mode': False,
#                       'wrapped_params': wrapped_params,
#                       'evidence_tolerance': 0.1,
#                       'max_iter': -1,
#                       'seed' : 0, # Fixing the seed
#                       'LHS_seed': 42, # Fixing the LHS seed for hypercube fraction estimation', 
#                       'verbose': True}

# xpsi.Sample.run_multinest(likelihood, prior, **runtime_params)

#print("Done ...")

#print('Sampling took', (time.time()-start)/60, 'minutes')